Task 2: Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
#Importar Data
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
table_fra=pd.DataFrame(tables[0])
columns_name = ['PostalCode','Borough','Neighborhood']
table_fra.columns = columns_name
table_fra.head()

ImportError: lxml not found, please install it

In [3]:
#Identificar 'Borough' cuando es 'Not assigned'
g=table_fra.loc[table_fra['Borough']=='Not assigned']
print("Borough like Not assigned is", g.shape)

NameError: name 'table_fra' is not defined

In [4]:
#Eliminar registros cuando 'Borough' es 'Not assigned'
table_fra.drop(g.index, inplace = True)


NameError: name 'table_fra' is not defined

In [5]:
#Identificar 'Neighborhood' cuando es 'Not assigned'
g2=table_fra.loc[table_fra['Neighborhood']=='Not assigned']
print("Neighbourhood like Not assigned",g2.shape)
#Identificar si un Codigo Postal tiene mas de un vecindario
g3=table_fra.groupby(['PostalCode'],as_index=False).count()
g3_1= g3.loc[g3['Neighborhood']==2]
print("More than one neighborhood can exist in one postal code area",g3_1.shape)

NameError: name 'table_fra' is not defined

In [6]:
#Table is clear 
table_fra.shape


NameError: name 'table_fra' is not defined

Task 2: 

In [7]:
import pandas as pd2
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')
Geospatial_data = pd2.read_csv('Geospatial_data.csv')
columns_name2 = ['PostalCode','Latitude','Longitude']
Geospatial_data.columns = columns_name2
Geospatial_data.head()

Data downloaded!


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
GeospatialwithTable= Geospatial_data[Geospatial_data['PostalCode'].isin(table_fra['PostalCode'].tolist())]
GeospatialwithTable.shape

NameError: name 'table_fra' is not defined

In [9]:
Data_Merge= pd.merge(table_fra, GeospatialwithTable, on='PostalCode')
Data_Merge.head()

NameError: name 'table_fra' is not defined

Task 3: Insight

In [10]:
#Q Borough
Data_group=Data_Merge.groupby(['Borough'],as_index=False).count()
Data_group

NameError: name 'Data_Merge' is not defined

In [11]:
Toronto_Data =Data_Merge[Data_Merge.Borough.str.contains('Toronto')]
Toronto_Data.head()

NameError: name 'Data_Merge' is not defined

In [12]:
! pip install folium==0.5.0
import folium # plotting library
!pip install geopy
from geopy.geocoders import Nominatim

address = 'Toronto, Canadá'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map

for lat, lng, borough, neighborhood in zip(Toronto_Data['Latitude'], Toronto_Data['Longitude'], Toronto_Data['Borough'], Toronto_Data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,color='blue',
        fill=True,fill_color='#3186cc',fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  

# display map
toronto_map

     |████████████████████████████████| 112kB 5.3MB/s eta 0:00:01
43.6534817 -79.3839347


NameError: name 'Toronto_Data' is not defined

In [ ]:
#C
import requests 
CLIENT_ID = 'LYQS0RPQHSSLCADD3411DNEF00KIT4D3GBER4GLP0Q1XDJQC' # your Foursquare ID
CLIENT_SECRET = 'KFIMJG100141HTEP1C2VHQESLIAPJXYD4AZ5RGDZO5GLEDX2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)   
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, 
            v['venue']['name'], v['venue']['location']['lat'],  v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 'Venue Latitude',  'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=Toronto_Data['Neighborhood'],
                                   latitudes=Toronto_Data['Latitude'],
                                   longitudes=Toronto_Data['Longitude'])
toronto_venues.head()

NameError: name 'getNearbyVenues' is not defined

In [14]:
# Top Categories to each 'Neighborhood'
Toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.shape

NameError: name 'toronto_venues' is not defined